## 使用输出解析器格式化输出

一种格式化模型输出的有用方法，通常用于结构化输出。

两个概念：

1. 格式说明：一个自动生成的提示，告诉 LLM 如何根据想要的结果格式化它的响应
2. 解析器：一种将模型的文本输出提取到所需结构（通常为 json）的方法

两种方式：

1. 设置提示词中说明。
2. 提取模型输出内容，转化为 json 格式

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

### 方法一：在提示词中说明

In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain import PromptTemplate

In [2]:
# 定义响应结构
response_schemas = [
    ResponseSchema(name='bad_string', description='这是一个格式不正确的用户输入字符串'),
    ResponseSchema(name='good_string', description='这是您的回复，重新格式化的回复')
]

# 你想如何解析你的输出
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 查看用于【输出格式化】的提示模板
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // 这是一个格式不正确的用户输入字符串
	"good_string": string  // 这是您的回复，重新格式化的回复
}
```


In [3]:
template="""
您将从用户那里得到一个格式不正确的字符串。重新格式化并确保所有单词拼写正确

{format_instructions}

% 用户输入:
{user_input}

你的响应:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions}
)

final_prompt = prompt.format(user_input='我骑车去了昨天上海')
print(final_prompt)


您将从用户那里得到一个格式不正确的字符串。重新格式化并确保所有单词拼写正确

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // 这是一个格式不正确的用户输入字符串
	"good_string": string  // 这是您的回复，重新格式化的回复
}
```

% 用户输入:
我骑车去了昨天上海

你的响应:



In [4]:
llm = OpenAI()

result = llm(final_prompt)
print(result)

```json
{
	"bad_string": "我骑车去了昨天上海",
	"good_string": "我昨天骑车去了上海"
}
```


In [5]:
type(result)

str

In [9]:
output_dict = output_parser.parse(result)

In [10]:
output_dict

{'bad_string': '我骑车去了昨天上海', 'good_string': '我昨天骑车去了上海'}

In [11]:
type(output_dict)

dict